### Joins
---------
The material in this notebook was extracted from
* Spark The Definitive Guide Big Data Processing Made Simple (2018)
---------

A join brings together two sets of data, the left and the right, by comparing the value of one or more *keys*: if they are equal, Spark will combine the left and right datasets. The opposite is true for keys that do not match; Spark discards the rows that do not have matching keys. 

##### Join Types

Whereas the join expression determines whether two rows should join, the join type determines what should be in the result set. There are various join types available in Spark:

* Inner joins (keep rows with keys that exist in the left and right datasets)
* Outer joins (keep rows with keys in either the left or right datasets)
* Left semi joins (keep the rows of the left dataset where the key appears in the right dataset)
* Left anti joins (keep the rows of the left dataset where the key doesn't appear in the right dataset)
* Natural joins (perform a join by implicitly matching the columns between the two datasets with the same names)
* Cross joins (match every row in the left dataset with every row in the right dataset)

In [0]:
person = spark.createDataFrame([
    (0, "Bill Chambers", 0, [100]),
    (1, "Matel Zaharia", 1, [500, 250, 100]),
    (2, "Michael Armbrust", 1, [250, 100]),
    (3, "Luis", 4, [250, 100, 1000])
]).toDF("id", "name", "graduate_program", "spark_status")
program = spark.createDataFrame([
    (0, "Masters", "School of Information", "UC Berkley"),
    (1, "Masters", "EECS", "UC Berkley"),
    (2, "Ph.D.", "EECS", "UC Berkley")
]).toDF("id", "degree", "department", "school")
status = spark.createDataFrame([
    (500, "Vice President"),
    (250, "PMC Member"),
    (100, "Contributor"),
    (1000, "CEO")
]).toDF("id", "status")

NameError: ignored

##### Inner Joins

Inner joins evaluate keys in both DataFrames and include only the rows that evaluate to true.

In [0]:
joinCondition = person['graduate_program'] == program['id']

In [0]:
person.join(program, joinCondition).show()

+---+----------------+----------------+---------------+---+-------+--------------------+----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|    school|
+---+----------------+----------------+---------------+---+-------+--------------------+----------+
|  0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkley|
|  1|   Matel Zaharia|               1|[500, 250, 100]|  1|Masters|                EECS|UC Berkley|
|  2|Michael Armbrust|               1|     [250, 100]|  1|Masters|                EECS|UC Berkley|
+---+----------------+----------------+---------------+---+-------+--------------------+----------+



Or more explicitly 

In [0]:
person.join(program, 
            person['graduate_program'] == program['id'], 
            'inner').show()

+---+----------------+----------------+---------------+---+-------+--------------------+----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|    school|
+---+----------------+----------------+---------------+---+-------+--------------------+----------+
|  0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkley|
|  1|   Matel Zaharia|               1|[500, 250, 100]|  1|Masters|                EECS|UC Berkley|
|  2|Michael Armbrust|               1|     [250, 100]|  1|Masters|                EECS|UC Berkley|
+---+----------------+----------------+---------------+---+-------+--------------------+----------+



##### Outer Joins

Outer joins evaluate the keys in both of the DataFrames and includes the rows that evaluate true or false. If there is no eequivalent row in either left or right DataFrame, Spark will insert null

In [0]:
person.join(program, 
            person['graduate_program'] == program['id'], 
            'outer').show()

+----+----------------+----------------+----------------+----+-------+--------------------+----------+
|  id|            name|graduate_program|    spark_status|  id| degree|          department|    school|
+----+----------------+----------------+----------------+----+-------+--------------------+----------+
|   0|   Bill Chambers|               0|           [100]|   0|Masters|School of Informa...|UC Berkley|
|   1|   Matel Zaharia|               1| [500, 250, 100]|   1|Masters|                EECS|UC Berkley|
|   2|Michael Armbrust|               1|      [250, 100]|   1|Masters|                EECS|UC Berkley|
|null|            null|            null|            null|   2|  Ph.D.|                EECS|UC Berkley|
|   3|            Luis|               4|[250, 100, 1000]|null|   null|                null|      null|
+----+----------------+----------------+----------------+----+-------+--------------------+----------+



In [0]:
person.join(program, 
            person['graduate_program'] == program['id'], 
            'left_outer').show()

+---+----------------+----------------+----------------+----+-------+--------------------+----------+
| id|            name|graduate_program|    spark_status|  id| degree|          department|    school|
+---+----------------+----------------+----------------+----+-------+--------------------+----------+
|  0|   Bill Chambers|               0|           [100]|   0|Masters|School of Informa...|UC Berkley|
|  1|   Matel Zaharia|               1| [500, 250, 100]|   1|Masters|                EECS|UC Berkley|
|  2|Michael Armbrust|               1|      [250, 100]|   1|Masters|                EECS|UC Berkley|
|  3|            Luis|               4|[250, 100, 1000]|null|   null|                null|      null|
+---+----------------+----------------+----------------+----+-------+--------------------+----------+



In [0]:
person.join(program, 
            person['graduate_program'] == program['id'], 
            'right_outer').show()

+----+----------------+----------------+---------------+---+-------+--------------------+----------+
|  id|            name|graduate_program|   spark_status| id| degree|          department|    school|
+----+----------------+----------------+---------------+---+-------+--------------------+----------+
|   0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkley|
|   1|   Matel Zaharia|               1|[500, 250, 100]|  1|Masters|                EECS|UC Berkley|
|   2|Michael Armbrust|               1|     [250, 100]|  1|Masters|                EECS|UC Berkley|
|null|            null|            null|           null|  2|  Ph.D.|                EECS|UC Berkley|
+----+----------------+----------------+---------------+---+-------+--------------------+----------+



##### Left Semi Joins

They do not actually include any values from the right DataFrame. If the value does exist, those rows will be kept in the result, even if there are duplicate keys in the left DataFrame. 

In [0]:
program.join(person, 
            person['graduate_program'] == program['id'], 
            'left_semi').show()

+---+-------+--------------------+----------+
| id| degree|          department|    school|
+---+-------+--------------------+----------+
|  0|Masters|School of Informa...|UC Berkley|
|  1|Masters|                EECS|UC Berkley|
+---+-------+--------------------+----------+



In [0]:
person.join(program, 
           person['graduate_program'] == program['id'],
           'left_semi').show()

+---+----------------+----------------+---------------+
| id|            name|graduate_program|   spark_status|
+---+----------------+----------------+---------------+
|  0|   Bill Chambers|               0|          [100]|
|  1|   Matel Zaharia|               1|[500, 250, 100]|
|  2|Michael Armbrust|               1|     [250, 100]|
+---+----------------+----------------+---------------+



##### Left Anti Joins

Left anti joins are the opposite of left semi joins.

In [0]:
program.join(person, 
            person['graduate_program'] == program['id'], 
            'left_anti').show()

+---+------+----------+----------+
| id|degree|department|    school|
+---+------+----------+----------+
|  2| Ph.D.|      EECS|UC Berkley|
+---+------+----------+----------+



##### Joins on Complex Types

Any expression is a valid join expression, assuming it returns a Boolean:

In [0]:
from pyspark.sql.functions import expr

person.selectExpr('id as personId', 'name', 'spark_status')\
.join(status, expr('array_contains(spark_status, id)')).show()

+--------+----------------+----------------+----+--------------+
|personId|            name|    spark_status|  id|        status|
+--------+----------------+----------------+----+--------------+
|       0|   Bill Chambers|           [100]| 100|   Contributor|
|       1|   Matel Zaharia| [500, 250, 100]| 500|Vice President|
|       1|   Matel Zaharia| [500, 250, 100]| 250|    PMC Member|
|       1|   Matel Zaharia| [500, 250, 100]| 100|   Contributor|
|       2|Michael Armbrust|      [250, 100]| 250|    PMC Member|
|       2|Michael Armbrust|      [250, 100]| 100|   Contributor|
|       3|            Luis|[250, 100, 1000]| 250|    PMC Member|
|       3|            Luis|[250, 100, 1000]| 100|   Contributor|
|       3|            Luis|[250, 100, 1000]|1000|           CEO|
+--------+----------------+----------------+----+--------------+

